In [ ]:
# import library
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import h5py 
from matplotlib.ticker import AutoLocator
from matplotlib.ticker import MaxNLocator
tf.random.set_seed(1)
import tqdm

In [ ]:
!nvidia-smi

Tue Jun 28 02:42:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Split and save dataset

In [ ]:
!gdown --id 1TbMAXAUicGOqh23OAbSpf3A2-Ct_yCZr

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1TbMAXAUicGOqh23OAbSpf3A2-Ct_yCZr
To: /content/1512427.zip
100% 880M/880M [00:12<00:00, 70.7MB/s]


In [ ]:
!unzip '/content/1512427.zip' -d '/content/dataset'

Archive:  /content/1512427.zip
 extracting: /content/dataset/brainTumorDataPublic_1-766.zip  
 extracting: /content/dataset/brainTumorDataPublic_1533-2298.zip  
 extracting: /content/dataset/brainTumorDataPublic_767-1532.zip  
 extracting: /content/dataset/brainTumorDataPublic_2299-3064.zip  
 extracting: /content/dataset/cvind.mat  
 extracting: /content/dataset/README.txt  


In [ ]:
!unzip '/content/dataset/brainTumorDataPublic_1-766.zip' -d '/content/dataset/brainTumorDataPublic_1-766'
!unzip '/content/dataset/brainTumorDataPublic_767-1532.zip' -d '/content/dataset/brainTumorDataPublic_767-1532'
!unzip '/content/dataset/brainTumorDataPublic_1533-2298.zip' -d '/content/dataset/brainTumorDataPublic_1533-2298'
!unzip '/content/dataset/brainTumorDataPublic_2299-3064.zip' -d '/content/dataset/brainTumorDataPublic_2299-3064'


In [ ]:
PATH = '/content/dataset'

In [ ]:
# list of all paths of images
list_ds = []
for roots, dirs, files in os.walk(PATH):
    for di in dirs:
        for im in os.listdir(os.path.join(PATH,di)):
            im_path = os.path.join(PATH, di, im)
            list_ds.append(im_path)
len(list_ds)

3064

In [ ]:
list_ds_tf = tf.data.Dataset.from_tensor_slices(list_ds)

In [ ]:
# check dataset size
DATASET_SIZE = tf.data.experimental.cardinality(list_ds_tf).numpy()
DATASET_SIZE

3064

In [ ]:
# split original dataset into three subsets: train, validation, test by the proportions of 10%, 10%, 80%, respectively
list_ds_shu = list_ds_tf.shuffle(DATASET_SIZE, seed = 1234, reshuffle_each_iteration = False)
train_size = int(0.1 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)
test_size = int(0.8 * DATASET_SIZE)
train_ds = list_ds_shu.take(train_size)
remaining = list_ds_shu.skip(train_size)
val_ds = remaining.take(val_size)
test_ds = remaining.skip(val_size)
print('The size of training set: ', tf.data.experimental.cardinality(train_ds).numpy())
print('The size of testing set: ',tf.data.experimental.cardinality(test_ds).numpy())
print('The size of validation set: ',tf.data.experimental.cardinality(val_ds).numpy())

The size of training set:  306
The size of testing set:  2452
The size of validation set:  306


In [ ]:
print(os.path.exists('/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/train'))
print(os.path.exists('/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/test'))
print(os.path.exists('/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/val'))

True
True
True


In [ ]:
# save train set
tf.data.experimental.save(train_ds, '/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/train')
#save val set
tf.data.experimental.save(val_ds, '/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/val')
# save test set
tf.data.experimental.save(test_ds, '/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/test')



In [ ]:
# load and check
check_train = tf.data.experimental.load('/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/train')
for i in check_train.take(5):
  print(i)

tf.Tensor(b'/content/dataset/brainTumorDataPublic_2299-3064/2440.mat', shape=(), dtype=string)
tf.Tensor(b'/content/dataset/brainTumorDataPublic_767-1532/1152.mat', shape=(), dtype=string)
tf.Tensor(b'/content/dataset/brainTumorDataPublic_767-1532/1253.mat', shape=(), dtype=string)
tf.Tensor(b'/content/dataset/brainTumorDataPublic_2299-3064/2741.mat', shape=(), dtype=string)
tf.Tensor(b'/content/dataset/brainTumorDataPublic_1-766/457.mat', shape=(), dtype=string)


In [ ]:
check_train = check_train.map(lambda x: tf.py_function(process_ds, [x], [tf.float32, tf.float16]))
check_train = check_train.map(fixed_shape)
check_train.element_spec

(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(3,), dtype=tf.float16, name=None))

In [ ]:
# load image and label from .mat file
def process_ds(file_path):
    source_file = h5py.File(file_path.numpy().decode('utf-8'), 'r')
    label_h5 = source_file.get('cjdata/label')
    label = int(np.asarray(label_h5)[0][0]) - 1
    image_h5 = source_file.get('cjdata/image')
    try:
        image = np.array(image_h5).reshape((512,512))
    except:
        image = np.array(image_h5).reshape((256,256))
    image_stacked = np.stack((image,image,image))
    image_stacked = np.moveaxis(image_stacked, 0, -1)

    return image_stacked, label

In [ ]:
# normalize and form the shape of each element
def fixed_shape(image_stacked, label):
  image_stacked.set_shape([224,224,3])
  image_stacked = tf.image.resize(image_stacked, [224, 224]) # resize original size of 512x512 to 224x224
  normalize = tf.keras.layers.Rescaling(1./255) # normalize the range of image value from [0,255] to [0,1]
  image_stacked_tf = normalize(image_stacked)
  image_stacked_tf.set_shape([224,224,3])
  label_one_hot = tf.one_hot(int(label), depth = 3, dtype = tf.float16) # classes number = 3
  label_one_hot.set_shape([3,])
  return image_stacked_tf, label_one_hot

In [ ]:
# mapping dataset
train_ds = train_ds.map(lambda x: tf.py_function(process_ds, [x], [tf.float32, tf.float16]))
train_ds = train_ds.map(fixed_shape)
val_ds = val_ds.map(lambda x: tf.py_function(process_ds, [x], [tf.float32, tf.float16]))
val_ds = val_ds.map(fixed_shape)
test_ds = test_ds.map(lambda x: tf.py_function(process_ds, [x], [tf.float32, tf.float16]))
test_ds = test_ds.map(fixed_shape)
# check element spec
train_ds.element_spec

(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(3,), dtype=tf.float16, name=None))

In [ ]:
count_0 = 0
count_1 = 0
count_2 = 0
for x,y in test_ds:
  if tf.argmax(y).numpy() == 0:
    count_0 += 1
  elif tf.argmax(y).numpy() == 1:
    count_1 += 1 
  elif tf.argmax(y).numpy() == 2:
    count_2 += 1
print('count 0: ', count_0)
print('count 1: ', count_1)
print('count 2: ', count_2)


count 0:  555
count 1:  1153
count 2:  744


# Load dataset 

In [ ]:
!gdown --id 1TbMAXAUicGOqh23OAbSpf3A2-Ct_yCZr

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1TbMAXAUicGOqh23OAbSpf3A2-Ct_yCZr
To: /content/1512427.zip
100% 880M/880M [00:12<00:00, 72.1MB/s]


In [ ]:
!unzip '/content/1512427.zip' -d '/content/dataset'

Archive:  /content/1512427.zip
 extracting: /content/dataset/brainTumorDataPublic_1-766.zip  
 extracting: /content/dataset/brainTumorDataPublic_1533-2298.zip  
 extracting: /content/dataset/brainTumorDataPublic_767-1532.zip  
 extracting: /content/dataset/brainTumorDataPublic_2299-3064.zip  
 extracting: /content/dataset/cvind.mat  
 extracting: /content/dataset/README.txt  


In [ ]:
!unzip '/content/dataset/brainTumorDataPublic_1-766.zip' -d '/content/dataset/brainTumorDataPublic_1-766'
!unzip '/content/dataset/brainTumorDataPublic_767-1532.zip' -d '/content/dataset/brainTumorDataPublic_767-1532'
!unzip '/content/dataset/brainTumorDataPublic_1533-2298.zip' -d '/content/dataset/brainTumorDataPublic_1533-2298'
!unzip '/content/dataset/brainTumorDataPublic_2299-3064.zip' -d '/content/dataset/brainTumorDataPublic_2299-3064'


In [ ]:
train_ds = tf.data.experimental.load('/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/train')
val_ds = tf.data.experimental.load('/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/val')
test_ds = tf.data.experimental.load('/content/drive/MyDrive/brain-tumor/dataset/save_splited_name/test')

In [ ]:
# load image and label from .mat file
def process_ds_gan(file_path):
    source_file = h5py.File(file_path.numpy().decode('utf-8'), 'r')
    label_h5 = source_file.get('cjdata/label')
    label = int(np.asarray(label_h5)[0][0]) - 1
    image_h5 = source_file.get('cjdata/image')
    try:
        image = np.array(image_h5).reshape((512,512))
        image = np.expand_dims(image, axis = -1)
    except:
        image = np.array(image_h5).reshape((256,256))
        image = np.expand_dims(image,axis = -1)
    #image_stacked = np.stack((image,image,image))
    #image_stacked = np.moveaxis(image_stacked, 0, -1)

    return image, label

In [ ]:
# normalize and form the shape of each element
def fixed_shape_gan(image_stacked, label):
  image_stacked.set_shape([224,224,1])
  image_stacked = tf.image.resize(image_stacked, [224, 224]) # resize original size of 512x512 to 224x224
  data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.2),
    ]
  ) 
  image_stacked = data_augmentation(image_stacked)
  normalize = tf.keras.layers.Rescaling(1./127.5, offset = -1) # normalize the range of image value from [0,255] to [0,1]
  image_stacked_tf = normalize(image_stacked)
  image_stacked_tf.set_shape([224,224,1])
  label_one_hot = tf.one_hot(int(label), depth = 3, dtype = tf.float16) # classes number = 3
  label_one_hot.set_shape([3,])
  return image_stacked_tf, label_one_hot

In [ ]:
# mapping dataset
train_ds = train_ds.map(lambda x: tf.py_function(process_ds_gan, [x], [tf.float32, tf.float16]))
train_ds = train_ds.map(fixed_shape_gan)
val_ds = val_ds.map(lambda x: tf.py_function(process_ds_gan, [x], [tf.float32, tf.float16]))
val_ds = val_ds.map(fixed_shape_gan)
test_ds = test_ds.map(lambda x: tf.py_function(process_ds_gan, [x], [tf.float32, tf.float16]))
test_ds = test_ds.map(fixed_shape_gan)
# check element spec
train_ds.element_spec

(TensorSpec(shape=(224, 224, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(3,), dtype=tf.float16, name=None))

In [ ]:
train_ds.element_spec

(TensorSpec(shape=(224, 224, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(3,), dtype=tf.float16, name=None))

In [ ]:
# shuffle dataset and divide it into batches 
def configure_for_performance(ds, batch_size):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size = 1024)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size = tf.data.AUTOTUNE)
    return ds

# GANs

## AC-GAN

In [ ]:

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply,  BatchNormalization, Activation, Embedding, ZeroPadding2D, Conv2DTranspose , Conv2D, LeakyReLU, UpSampling2D, Concatenate, MaxPool2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def define_generator(latent_dim, n_classes=3):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # label input
  in_label = Input(shape=(3,))
  # embedding for categorical input
  li = Embedding(n_classes, 50)(in_label)
  # linear multiplication
  n_nodes = 7 * 7
  li = Dense(n_nodes, kernel_initializer=init)(li)
  # reshape to additional channel
  li = Reshape((7, 7, 3))(li)
  # image generator input
  in_lat = Input(shape=(latent_dim,))
  # foundation for 7x7 image
  n_nodes = 384 * 7 * 7
  gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
  gen = Activation('relu')(gen)
  gen = Reshape((7, 7, 384))(gen)
  # merge image gen and label input
  merge = Concatenate()([gen, li])
  # upsample to 14x14
  gen = Conv2DTranspose(192, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merge)
  gen = BatchNormalization()(gen) 
  gen = Activation('relu')(gen)
    # upsample to 28x28
  gen = Conv2DTranspose(96, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  gen = BatchNormalization()(gen) 
  gen = Activation('relu')(gen)
    # upsample to 56x56
  gen = Conv2DTranspose(48, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  gen = BatchNormalization()(gen) 
  gen = Activation('relu')(gen)
   # upsample to 112x112
  gen = Conv2DTranspose(24, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  gen = BatchNormalization()(gen) 
  gen = Activation('relu')(gen)
  # upsample to 224x224
  gen = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  out_layer = Activation('tanh')(gen)
  # define model
  model = Model([in_lat, in_label], out_layer)
  return model
# define the size of the latent space
latent_dim = 100
# define the generator model
model = define_generator(latent_dim)
# summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 dense_1 (Dense)                (None, 18816)        1900416     ['input_2[0][0]']                
                                                                                                  
 embedding (Embedding)          (None, 3, 50)        150         ['input_1[0][0]']                
                                                                                              

In [ ]:
def define_discriminator(in_shape=(224,224,1), n_classes=3):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # image input
  in_image = Input(shape=in_shape)
  # downsample to 112x112
  fe = Conv2D(3, (3,3), strides=(4,4), padding='same', kernel_initializer=init)(in_image) 
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  # normal
  fe = Conv2D(16, (3,3), padding='same', kernel_initializer=init)(fe) 
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
    # downsample to 56x56
  fe = Conv2D(32, (3,3), strides=(4,4), padding='same', kernel_initializer=init)(fe) 
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  # normal
  fe = Conv2D(64, (3,3), padding='same', kernel_initializer=init)(fe) 
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  # downsample to 28x28
  fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe) 
  fe = BatchNormalization()(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  # normal
  # fe = Conv2D(224, (3,3), padding='same', kernel_initializer=init)(fe) 
  # fe = BatchNormalization()(fe)
  # fe = LeakyReLU(alpha=0.2)(fe)
  # fe = Dropout(0.5)(fe)
  # # downsample to 14x14
  # fe = Conv2D(512, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe) 
  # fe = BatchNormalization()(fe)
  # fe = LeakyReLU(alpha=0.2)(fe)
  # fe = Dropout(0.5)(fe)
  # # normal
  # fe = Conv2D(1024, (3,3), padding='same', kernel_initializer=init)(fe)
  # fe = MaxPool2D(pool_size = (4,4), strides = (2,2), padding = 'same')(fe)
  # fe = BatchNormalization()(fe)
  # fe = LeakyReLU(alpha=0.2)(fe)
  fe = Dropout(0.5)(fe)
  # flatten feature maps
  fe = Flatten()(fe)
  # real/fake output
  out1 = Dense(1, activation='sigmoid')(fe)
  # class label output
  out2 = Dense(n_classes, activation='softmax')(fe)
  # define model
  model = Model(in_image, [out1, out2])
  # compile model
  opt = Adam(learning_rate=0.0001, beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], loss_weights= [1, 1],
        optimizer=opt, metrics = ['accuracy'])
  return model

In [ ]:
dis = define_discriminator()
dis.metrics_names

[]

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  # connect the outputs of the generator to the inputs of the discriminator
  gan_output = d_model(g_model.output)
  # define gan model as taking noise and label and outputting real/fake and label outputs 
  model = Model(g_model.input, gan_output)
  # compile model
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], loss_weights= [1,1],
        optimizer=opt)
  return model

In [ ]:
def sample_images(gen_model, epoch, latent_dim):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, latent_dim))

        noise_labels = np.random.randint(0,3, (c*r,1))
        noise_labels_o = np.eye(3)[noise_labels]
        noise_labels_o = np.reshape(noise_labels_o, (c*r,3))

        gen_imgs = gen_model.predict([noise, noise_labels_o])

        # Rescale images 0 - 1
        gen_imgs = gen_imgs * 127.5 + 127.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/drive/MyDrive/brain-tumor/GAN/AC-GAN/3/images_equal_weights/%d.png" % epoch)
        plt.close()

In [ ]:
def save(model, model_name, epoch):
            model_path = "/content/drive/MyDrive/brain-tumor/GAN/AC-GAN/3/saved_model_equal_weights/%s_%d.json" % (model_name, epoch)
            weights_path = "/content/drive/MyDrive/brain-tumor/GAN/AC-GAN/3/saved_model_equal_weights/%s_weights_%d.hdf5" % (model_name, epoch)
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])


In [ ]:
def train(gan_model, g_model, d_model, train_ds, initial_epoch, total_epochs, batch_size, sample_interval, latent_dim):\

  train_ds = configure_for_performance(train_ds, batch_size)

  # initialize label for GAN
  constant_batch_size = batch_size
  for epoch in range(initial_epoch, total_epochs):
    print('Epoch:{}\n'.format(epoch+1))
    for step, (image_reals, labels) in enumerate(train_ds):
      if step == len(list(enumerate(train_ds))) - 1:
        batch_size = 306 - batch_size * step
      else:
        batch_size = constant_batch_size
      noise = np.random.normal(0,1, (batch_size, latent_dim))
      noise_labels = np.random.randint(0,3, (batch_size,1))
      noise_labels_o = np.eye(3)[noise_labels]
      noise_labels_o = np.reshape(noise_labels_o, (batch_size,3))

      valid = np.ones((batch_size, 1))
      fake = np.zeros((batch_size, 1))
      # generate fake image
      image_fake = g_model.predict([noise, noise_labels_o])
      #print(labels.shape)
      # train the discriminator
      d_loss_real = d_model.train_on_batch(image_reals, [valid, labels], return_dict = True)
      d_loss_fake = d_model.train_on_batch(image_fake, [fake, noise_labels_o], return_dict = True)


      # train the generator
      g_loss = gan_model.train_on_batch([noise, noise_labels_o], [valid, noise_labels_o], return_dict = True)
      print('d_loss_real: ' + str(d_loss_real) + '\n')
      print('d_loss_fake: ' + str(d_loss_fake) + '\n')
      print('gan_loss: ' + str(g_loss) + '\n')

    if epoch % sample_interval == 0:
      sample_images(g_model, epoch+1, latent_dim)
      save(g_model, 'generator', epoch+1)
      save(d_model, 'discriminator', epoch+1)



  


In [ ]:
print('%s_%d' % ('a', 3))

a_3


### Setting parameter and summary

In [ ]:
latent_dim = 100
batch_size = 64
initial_epoch = 0
total_epochs = 500
sample_interval = 20

In [ ]:
dis_model = define_discriminator()
gen_model = define_generator(latent_dim)
gan_model = define_gan(gen_model, dis_model)


In [ ]:
dis_model.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 56, 56, 3)    30          ['input_9[0][0]']                
                                                                                                  
 leaky_re_lu_10 (LeakyReLU)     (None, 56, 56, 3)    0           ['conv2d_10[0][0]']              
                                                                                                  
 dropout_12 (Dropout)           (None, 56, 56, 3)    0           ['leaky_re_lu_10[0][0]']   

In [ ]:
gen_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 dense_9 (Dense)                (None, 18816)        1900416     ['input_8[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, 3, 50)        150         ['input_7[0][0]']                
                                                                                            

In [ ]:
gan_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 dense_9 (Dense)                (None, 18816)        1900416     ['input_8[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, 3, 50)        150         ['input_7[0][0]']                
                                                                                            

### Training

In [ ]:
train(gan_model = gan_model, g_model = gen_model, d_model = dis_model, train_ds = train_ds, initial_epoch = initial_epoch,
      total_epochs = total_epochs, batch_size = batch_size, sample_interval = sample_interval, latent_dim = latent_dim)

Streaming output truncated to the last 5000 lines.
gan_loss: {'loss': 0.18159079551696777, 'model_31_loss': 0.09753678739070892, 'model_31_1_loss': 0.08405400067567825}

d_loss_real: {'loss': 0.5251775979995728, 'dense_42_loss': 0.008571965619921684, 'dense_43_loss': 0.5166056156158447, 'dense_42_accuracy': 1.0, 'dense_43_accuracy': 0.7799999713897705}

d_loss_fake: {'loss': 0.009434806182980537, 'dense_42_loss': 0.007804811000823975, 'dense_43_loss': 0.0016299948329105973, 'dense_42_accuracy': 1.0, 'dense_43_accuracy': 1.0}

gan_loss: {'loss': 0.23575539886951447, 'model_31_loss': 0.1173570528626442, 'model_31_1_loss': 0.11839836090803146}

Epoch:345

d_loss_real: {'loss': 0.6193248629570007, 'dense_42_loss': 0.013556525111198425, 'dense_43_loss': 0.6057683229446411, 'dense_42_accuracy': 1.0, 'dense_43_accuracy': 0.75}

d_loss_fake: {'loss': 0.007290496490895748, 'dense_42_loss': 0.004964169580489397, 'dense_43_loss': 0.002326326910406351, 'dense_42_accuracy': 1.0, 'dense_43_accuracy'

In [ ]:
noise_labels = np.random.randint(0,3, (16,1))
noise_labels_o = np.eye(3)[noise_labels]
noise_labels_o.shape

(16, 1, 3)

In [ ]:
noise_labels_o

array([[[0., 1., 0.]],

       [[1., 0., 0.]],

       [[0., 1., 0.]],

       [[0., 0., 1.]],

       [[0., 0., 1.]],

       [[1., 0., 0.]],

       [[0., 1., 0.]],

       [[1., 0., 0.]],

       [[1., 0., 0.]],

       [[0., 0., 1.]],

       [[0., 0., 1.]],

       [[1., 0., 0.]],

       [[0., 0., 1.]],

       [[1., 0., 0.]],

       [[0., 1., 0.]],

       [[0., 1., 0.]]])

In [ ]:
a = np.reshape(noise_labels_o, (16,3))

In [ ]:
a.shape

(16, 3)

In [ ]:
b= np.reshape(b, (-1,1))
b.shape

(48, 1)

In [ ]:
train_batch = configure_for_performance(train_ds,16)
print(len(list(enumerate(train_batch))))
for step, (x,y) in enumerate(train_batch):
  print(step)

20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
